In [34]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request
import time
import random

In [35]:
# 获取歌曲名称
song_names = pd.read_csv("all_songs.csv")['songs'].to_list()
singer_name = "陈奕迅"

In [36]:
# 检查用
len(song_names)

720

In [44]:
song_infos = []

url_baike = "https://baike.baidu.com"

# 四种情况
# case 1:正确找到对应歌曲百科
# case 2:找到多义词的百科界面-->需跳转到多义词的列表界面-->找到对应歌曲百科
# case 3:找到多义词的列表界面-->找到对应歌曲百科
# case 4:找不到百科

# 找到资料的前一个a标签
def is_info_pre_a(tag):
    return tag.has_attr('name') and tag['name'] == 'lemma-summary'

pattern = r'^共\d+个义项'
# 若有多义词，则必存在指向多义词的a标签
def is_multi_a(tag):
    return tag.has_attr('href') and re.search(pattern=pattern, string=tag.text)

# 从多义词列表中找到歌曲链接
def is_target_a(tag):
    return tag.has_attr('href') and singer_name in tag.text

# case 1
def get_baike(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "lxml")
    try:
        d = soup.find(is_info_pre_a).next_sibling
        if singer_name in d.get_text():
            song_infos.append([name, d.get_text()])
            print("添加歌曲信息成功")
        else:
            # case 2
            d = soup.find(is_multi_a)
            res = requests.get(url=url_baike+d['href'])
            soup = BeautifulSoup(res.text, "lxml")
            # 进入case 3
            d = soup.find(is_target_a)
            if d is None:
                # 说明找不到该歌曲对应百科
                print("找不到对应百科")
            else:
                res = requests.get(url=url_baike+d['href'])
                soup = BeautifulSoup(res.text, "lxml")

                d = soup.find(is_info_pre_a).next_sibling
                song_infos.append([name, d.get_text()])
                print("添加歌曲信息成功")
    except:
        # case 3 or case 4
        d = soup.find(is_target_a)
        if d is None:
            # 说明找不到该歌曲对应百科
            print("找不到对应百科")
        else:
            res = requests.get(url=url_baike+d['href'])
            soup = BeautifulSoup(res.text, "lxml")

            d = soup.find(is_info_pre_a).next_sibling
            song_infos.append([name, d.get_text()])
            print("添加歌曲信息成功")

num = 1
for name in song_names:
    print("["+str(num) + "/" + str(len(song_names))+"] "+name+":")
    get_baike(url=url_baike+"/item/"+name)

    num += 1
    n = random.randint(0, 10)
    time.sleep(n * 0.1)

[1/720] 不期而遇的夏天:
找不到对应百科
[2/720] 是但求其爱:
添加歌曲信息成功
[3/720] 致明日的舞:
添加歌曲信息成功
[4/720] Fight as ONE:
添加歌曲信息成功
[5/720] 相信你的人:
添加歌曲信息成功
[6/720] 破坏王:
找不到对应百科
[7/720] 海里睡人:
添加歌曲信息成功
[8/720] 渐渐:
找不到对应百科
[9/720] 敬烟:
添加歌曲信息成功
[10/720] 蠢:
找不到对应百科
[11/720] 疯狂的朋友:
添加歌曲信息成功
[12/720] R U N:
添加歌曲信息成功
[13/720] 龙舌兰:
找不到对应百科
[14/720] 我们万岁:
找不到对应百科
[15/720] 与你常在:
添加歌曲信息成功
[16/720] 可一可再:
添加歌曲信息成功
[17/720] 龙舌兰:
找不到对应百科
[18/720] 可一可再:
添加歌曲信息成功
[19/720] 渐渐:
找不到对应百科
[20/720] 我们:
找不到对应百科
[21/720] 放:
找不到对应百科
[22/720] 收心操:
添加歌曲信息成功
[23/720] 海胆:
找不到对应百科
[24/720] 谁来剪月光:
添加歌曲信息成功
[25/720] 床上的黑洞:
添加歌曲信息成功
[26/720] 右上角:
添加歌曲信息成功
[27/720] 之外:
添加歌曲信息成功
[28/720] 傅科摆:
添加歌曲信息成功
[29/720] 零下几分钟:
添加歌曲信息成功
[30/720] 披风:
找不到对应百科
[31/720] 海胆:
找不到对应百科
[32/720] 谁来剪月光:
添加歌曲信息成功
[33/720] 披风:
找不到对应百科
[34/720] 放:
找不到对应百科
[35/720] 放:
找不到对应百科
[36/720] 披风:
找不到对应百科
[37/720] 让我留在你身边:
找不到对应百科
[38/720] 四季:
找不到对应百科
[39/720] I Do:
找不到对应百科
[40/720] 在这个世界相遇:
添加歌曲信息成功
[41/720] Sleep Alone:
添加歌曲信息成功
[42/720] 陪你度过漫长岁月:
添加歌曲信息成功
[43/720] 恐龙进化论:
添加歌曲信息成功

In [45]:
len(song_infos)

36

In [46]:
# 将歌曲信息添加进文件中保存
df = pd.DataFrame(columns=["歌曲名称","歌曲信息"], data=song_infos)
df.to_csv("song_infos.csv")

In [47]:
df

,歌曲名称,歌曲信息
0,是但求其爱,《是但求其爱》是由小克作词，林家谦作曲、编曲 [2]， 陈奕迅演唱的粤语歌曲 [2]，由环球...
1,致明日的舞,《致明日的舞》是由Oscar作词，林家谦作曲，陈奕迅演唱的歌曲，2020年10月21日通过单...
2,Fight as ONE,《Fight As One》是由朱敬然、郭伟亮、陈哲庐作词，宋秉洋作曲，陈奕迅、蔡依林演唱的...
3,相信你的人,《相信你的人》是电影《银河补习班》推广曲，由姚若龙作词，陈小霞作曲，陈奕迅演唱，于2019年...
4,海里睡人,《海里睡人》是由卢凯彤作词，孙伟明、卢凯彤作曲，陈奕迅演唱的歌曲，收录于2018年12月12...
5,敬烟,《敬烟》是由陈咏谦作词，苏德华作曲，陈奕迅演唱的歌曲，收录于2018年12月12日发行的专辑...
6,疯狂的朋友,《疯狂的朋友》是由陈咏谦作词，恭硕良作曲，the duo band编曲，陈奕迅演唱的歌曲，收...
7,R U N,《R U N》是由岑宁儿作词、作曲，The Duo Band编曲，陈奕迅演唱的歌曲，收录于2...
8,与你常在,《与你常在》是由陈咏谦作词，Charles Arthur Huntley、Mark Jame...
9,可一可再,《可一可再》是由陈咏谦作词，陈奕迅作曲并演唱的歌曲，于2018年8月24日通过单曲形式发行 ...
